# _STAT 201 Group Project Proposal_

##### _Group 31: Alexandr Kim, Yingshan Wu, Yang Lei, Harbor Zhang_

### Adding necessary libraries

In order to load the packages, we have to use the code `install.packages()`

In [ ]:
install.packages("tidyverse")
install.packages("tidymodels")
install.packages("repr")
install.packages("cowplot")
install.packages("ggplot2")
install.packages("GGally")
install.packages("digest")
install.packages("infer")

After installing relevant packages, we need to load it into workflow

In [2]:
library(tidyverse)
library(cowplot)
library(infer)
library(repr)
library(ggplot2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



### Appendix A: Data set for street trees in Vancouver (BC, Canada) recorded by City of Vancouver